In [ ]:
# FOR LOCAL DEVELOPMENT, NOT NEEDED ON DATABRICKS
import os
import pathlib
from dotenv import load_dotenv
load_dotenv()

LOCAL_DATABRICKS_NOTEBOOK_PATH = os.getenv('LOCAL_DATABRICKS_NOTEBOOK_PATH')
if LOCAL_DATABRICKS_NOTEBOOK_PATH and pathlib.Path(LOCAL_DATABRICKS_NOTEBOOK_PATH).exists():
    print(f"Installing databricks-notebook from {LOCAL_DATABRICKS_NOTEBOOK_PATH}")
    %pip install --editable "{LOCAL_DATABRICKS_NOTEBOOK_PATH}"
    
    # Restart to make dependencies available
    import IPython
    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

In [ ]:
# Install dependencies
%pip install -qq --force-reinstall --no-cache-dir git+https://github.com/datafold/databricks-notebook.git

# Restart to make dependencies available
%restart_python

In [ ]:
from databricks_notebook import translate_queries_and_render_results

queries = [
"""
SELECT 
    o.order_id,
    o.customer_id,
    o.order_date,
    o.total_amount,
    c.customer_name,
    c.customer_segment,
    PARSE_JSON(o.order_metadata):shipping_address::STRING AS shipping_address,
    PARSE_JSON(o.order_metadata):payment_method::STRING AS payment_method,
    ROW_NUMBER() OVER (PARTITION BY o.customer_id ORDER BY o.order_date DESC) AS order_rank
FROM raw_sales.orders o
JOIN raw_customer.customers c ON o.customer_id = c.customer_id
WHERE o.order_date >= DATEADD(month, -6, CURRENT_DATE())
    AND o.status = 'COMPLETED'
QUALIFY order_rank <= 5
ORDER BY o.customer_id, o.order_date DESC
""",

"""
CREATE TABLE analytics.daily_product_metrics AS
SELECT 
    product_id,
    DATE_TRUNC('day', sale_timestamp) AS sale_date,
    COUNT(DISTINCT order_id) AS order_count,
    SUM(quantity) AS total_quantity_sold,
    SUM(net_revenue) AS total_revenue,
    AVG(unit_price) AS avg_unit_price,
    MEDIAN(unit_price) AS median_unit_price,
    COUNT(DISTINCT customer_id) AS unique_customers
FROM raw_sales.order_line_items
WHERE sale_timestamp >= DATEADD(year, -1, CURRENT_DATE())
GROUP BY product_id, DATE_TRUNC('day', sale_timestamp)
""",

"""
WITH customer_orders AS (
    SELECT 
        customer_id,
        COUNT(DISTINCT order_id) AS total_orders,
        SUM(order_total) AS lifetime_value,
        MIN(order_date) AS first_order_date,
        MAX(order_date) AS last_order_date
    FROM raw_sales.orders
    WHERE order_status NOT IN ('CANCELLED', 'REFUNDED')
    GROUP BY customer_id
),
customer_categories AS (
    SELECT 
        co.*,
        CASE 
            WHEN total_orders >= 20 THEN 'VIP'
            WHEN total_orders >= 10 THEN 'FREQUENT'
            WHEN total_orders >= 5 THEN 'REGULAR'
            ELSE 'OCCASIONAL'
        END AS customer_tier,
        DATEDIFF(day, first_order_date, last_order_date) AS customer_lifespan_days
    FROM customer_orders co
)
SELECT 
    customer_tier,
    COUNT(*) AS customer_count,
    AVG(lifetime_value) AS avg_lifetime_value,
    AVG(total_orders) AS avg_orders,
    AVG(customer_lifespan_days) AS avg_lifespan_days
FROM customer_categories
GROUP BY customer_tier
ORDER BY avg_lifetime_value DESC
""",

"""
SELECT 
    store_id,
    store_name,
    f.value:product_id::VARCHAR AS product_id,
    f.value:quantity::NUMBER AS quantity,
    f.value:last_updated::TIMESTAMP AS inventory_timestamp
FROM raw_operations.store_inventory,
LATERAL FLATTEN(input => PARSE_JSON(inventory_json)) f
WHERE f.value:quantity::NUMBER < 10
    AND store_region = 'WEST'
ORDER BY store_id, inventory_timestamp DESC
""",

"""
CREATE TABLE analytics.customer_cohort_analysis AS
WITH first_purchase AS (
    SELECT 
        customer_id,
        DATE_TRUNC('month', MIN(order_date)) AS cohort_month
    FROM raw_sales.orders
    WHERE order_status = 'COMPLETED'
    GROUP BY customer_id
),
monthly_activity AS (
    SELECT 
        fp.customer_id,
        fp.cohort_month,
        DATE_TRUNC('month', o.order_date) AS activity_month,
        SUM(o.order_total) AS monthly_revenue
    FROM first_purchase fp
    JOIN raw_sales.orders o ON fp.customer_id = o.customer_id
    WHERE o.order_status = 'COMPLETED'
    GROUP BY fp.customer_id, fp.cohort_month, DATE_TRUNC('month', o.order_date)
)
SELECT 
    cohort_month,
    activity_month,
    DATEDIFF(month, cohort_month, activity_month) AS months_since_first_purchase,
    COUNT(DISTINCT customer_id) AS active_customers,
    SUM(monthly_revenue) AS cohort_revenue
FROM monthly_activity
GROUP BY cohort_month, activity_month
""",

"""
WITH product_sales_7day AS (
    SELECT 
        product_id,
        SUM(quantity) AS qty_sold_7d,
        SUM(net_revenue) AS revenue_7d
    FROM raw_sales.order_line_items
    WHERE sale_timestamp >= DATEADD(day, -7, CURRENT_TIMESTAMP())
    GROUP BY product_id
),
product_sales_30day AS (
    SELECT 
        product_id,
        SUM(quantity) AS qty_sold_30d,
        SUM(net_revenue) AS revenue_30d
    FROM raw_sales.order_line_items
    WHERE sale_timestamp >= DATEADD(day, -30, CURRENT_TIMESTAMP())
    GROUP BY product_id
)
SELECT 
    p.product_id,
    p.product_name,
    p.category,
    ps7.qty_sold_7d,
    ps30.qty_sold_30d,
    ps7.revenue_7d,
    ps30.revenue_30d,
    RATIO_TO_REPORT(ps7.revenue_7d) OVER () AS pct_total_revenue_7d
FROM raw_product.products p
LEFT JOIN product_sales_7day ps7 ON p.product_id = ps7.product_id
LEFT JOIN product_sales_30day ps30 ON p.product_id = ps30.product_id
WHERE ps7.qty_sold_7d > 0 OR ps30.qty_sold_30d > 0
ORDER BY ps7.revenue_7d DESC NULLS LAST
""",

"""
SELECT 
    DATE_TRUNC('hour', event_timestamp) AS event_hour,
    event_type,
    COUNT(*) AS event_count,
    COUNT(DISTINCT session_id) AS unique_sessions,
    COUNT(DISTINCT user_id) AS unique_users,
    AVG(PARSE_JSON(event_properties):duration_seconds::NUMBER) AS avg_duration,
    ARRAY_AGG(DISTINCT PARSE_JSON(event_properties):page_url::STRING) 
        WITHIN GROUP (ORDER BY event_timestamp) AS visited_pages
FROM raw_web.clickstream_events
WHERE event_timestamp >= DATEADD(day, -1, CURRENT_TIMESTAMP())
    AND event_type IN ('PAGE_VIEW', 'ADD_TO_CART', 'CHECKOUT', 'PURCHASE')
GROUP BY DATE_TRUNC('hour', event_timestamp), event_type
HAVING event_count > 100
ORDER BY event_hour DESC, event_count DESC
""",

"""
CREATE TABLE analytics.store_performance_weekly AS
SELECT 
    s.store_id,
    s.store_name,
    s.region,
    DATE_TRUNC('week', o.order_date) AS week_start_date,
    COUNT(DISTINCT o.order_id) AS weekly_orders,
    COUNT(DISTINCT o.customer_id) AS weekly_customers,
    SUM(o.order_total) AS weekly_revenue,
    AVG(o.order_total) AS avg_order_value,
    LAG(SUM(o.order_total), 1) OVER (PARTITION BY s.store_id ORDER BY DATE_TRUNC('week', o.order_date)) AS prev_week_revenue,
    (SUM(o.order_total) - LAG(SUM(o.order_total), 1) OVER (PARTITION BY s.store_id ORDER BY DATE_TRUNC('week', o.order_date))) 
        / NULLIF(LAG(SUM(o.order_total), 1) OVER (PARTITION BY s.store_id ORDER BY DATE_TRUNC('week', o.order_date)), 0) AS revenue_growth_rate
FROM raw_operations.stores s
JOIN raw_sales.orders o ON s.store_id = o.store_id
WHERE o.order_date >= DATEADD(month, -3, CURRENT_DATE())
GROUP BY s.store_id, s.store_name, s.region, DATE_TRUNC('week', o.order_date)
""",

"""
WITH returns_analysis AS (
    SELECT 
        r.return_id,
        r.order_id,
        r.return_date,
        o.order_date,
        DATEDIFF(day, o.order_date, r.return_date) AS days_to_return,
        r.return_reason,
        oli.product_id,
        oli.quantity AS ordered_quantity,
        r.returned_quantity,
        oli.net_revenue AS original_revenue
    FROM raw_sales.returns r
    JOIN raw_sales.orders o ON r.order_id = o.order_id
    JOIN raw_sales.order_line_items oli ON r.order_id = oli.order_id AND r.product_id = oli.product_id
    WHERE r.return_date >= DATEADD(month, -6, CURRENT_DATE())
)
SELECT 
    product_id,
    return_reason,
    COUNT(DISTINCT return_id) AS return_count,
    SUM(returned_quantity) AS total_returned_qty,
    SUM(original_revenue * (returned_quantity / ordered_quantity)) AS revenue_impact,
    AVG(days_to_return) AS avg_days_to_return,
    PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY days_to_return) AS median_days_to_return
FROM returns_analysis
GROUP BY product_id, return_reason
QUALIFY ROW_NUMBER() OVER (PARTITION BY product_id ORDER BY return_count DESC) <= 3
ORDER BY product_id, return_count DESC
""",

"""
SELECT 
    c.customer_id,
    c.customer_name,
    c.email,
    c.signup_date,
    COUNT(DISTINCT o.order_id) AS total_orders,
    SUM(o.order_total) AS total_spent,
    ARRAY_AGG(DISTINCT p.category) WITHIN GROUP (ORDER BY p.category) AS purchased_categories,
    OBJECT_AGG(p.category, COUNT(DISTINCT o.order_id)) AS orders_by_category,
    MAX(o.order_date) AS last_order_date,
    DATEDIFF(day, MAX(o.order_date), CURRENT_DATE()) AS days_since_last_order,
    IFF(DATEDIFF(day, MAX(o.order_date), CURRENT_DATE()) > 90, TRUE, FALSE) AS is_at_risk
FROM raw_customer.customers c
LEFT JOIN raw_sales.orders o ON c.customer_id = o.customer_id
LEFT JOIN raw_sales.order_line_items oli ON o.order_id = oli.order_id
LEFT JOIN raw_product.products p ON oli.product_id = p.product_id
WHERE c.signup_date >= DATEADD(year, -2, CURRENT_DATE())
    AND o.order_status = 'COMPLETED'
GROUP BY c.customer_id, c.customer_name, c.email, c.signup_date
HAVING total_spent > 500
ORDER BY total_spent DESC
"""
]

translate_queries_and_render_results(queries)